### Introduction to Machine Learning, UZH FS18, Group Project

### Group 2: Barbara Capl, Mathias Lüthi, Pamela Matias, Stefanie Rentsch


#     
# III.     Feature Exraction with PCA

In [1]:
# hide unnecessary warnings ("depreciation" of packages etc.)
import warnings
warnings.filterwarnings('ignore')

# import packages
import numpy as np
import pandas as pd
import matplotlib as pl
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import datetime as dt
import sklearn as skl
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn import neural_network
from sklearn import neighbors
from sklearn.svm import SVR
from sklearn import neighbors
from functools import reduce
from functools import reduce
from pandas.core import datetools

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.tree import DecisionTreeClassifier

# Import datasets


In [2]:
# Import imputed dataset
imputed_dataset = pd.read_csv('Data/generated/imputed_dataset_ml.csv', sep = ',')
# Import dataset wit dropped Nans
dropnan_dataset = pd.read_csv('Data/generated/dropnan_dataset_ml.csv', sep = ',')

print('Shape of Imputed Dataset = ' + str(imputed_dataset.shape))
print('Shape of Dataset with Nans dropped = ' + str(dropnan_dataset.shape))



Shape of Imputed Dataset = (3519, 94)
Shape of Dataset with Nans dropped = (1430, 94)


# Prepare Data for Version 1: Imputed Dataset

### Feature Matrix and Response Vector 

In [3]:
# Extract labels of features
labels_of_features_1 = imputed_dataset.columns[:-1]
type(labels_of_features_1)

# X1 is the feature matrix
X1 = imputed_dataset.iloc[:, :-1]

display(X1.head())

,Unnamed: 0,PERMNO,DATE,NAICS,BIDLO,ASKHI,PRC,VOL,BID,ASK,...,sale_nwc,rd_sale,adv_sale,staff_sale,accrual,ptb,PEG_trailing,divyield,PEG_1yrforward,PEG_ltgforward
0,1,10107.0,1.138752e+18,511210.0,26.39,28.0400,26.87,11088149.0,26.87,26.88,...,1.323,0.151,0.025,0.0,0.036,6.281,10.280,0.0134,14.555,1.838
1,2,10107.0,1.141171e+18,511210.0,26.85,27.8900,27.21,14514337.0,27.24,27.24,...,1.323,0.151,0.025,0.0,0.036,6.293,10.410,0.0132,14.739,1.842
2,3,10107.0,1.143850e+18,511210.0,24.15,27.7400,24.15,14689919.0,24.16,24.16,...,1.323,0.151,0.025,0.0,0.036,5.573,9.239,0.0149,13.081,1.666
3,4,10107.0,1.146442e+18,511210.0,22.56,24.2900,22.65,23651189.0,22.70,22.70,...,1.388,0.150,0.025,0.0,0.024,5.496,0.709,0.0159,-5.842,1.480
4,5,10107.0,1.149120e+18,511210.0,21.51,23.4702,23.30,19980809.0,23.38,23.31,...,1.388,0.150,0.025,0.0,0.024,5.577,0.730,0.0155,-6.010,1.522


In [4]:
# y1 is the response vector
y1 = imputed_dataset.iloc[:, -1]
display(y1.head())

0    1.0
1    0.0
2    0.0
3    1.0
4    1.0
Name: NEXT_DAY_PREDICTION, dtype: float64

### Train - / Test - Split

In [5]:
# Do the train - test- split
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.2, random_state = 0, stratify = y1)


In [6]:
# Check if there is the approximately same percentage of '1' i both training and test response vector
display(y1_train.sum() / y1_train.size)
display(y1_test.sum() / y1_test.size)


0.5602131438721136

0.5610795454545454

### Standardize Variables

In [7]:
# Standardization with sklearn StandardScaler
standard_scaler_1 = preprocessing.StandardScaler().fit(X1_train)
X1_train = standard_scaler_1.transform(X1_train)
X1_test = standard_scaler_1.transform(X1_test)


In [16]:
# PIPELINE
# # https://stats.stackexchange.com/questions/144439/applying-pca-to-test-data-for-classification-purposes


# predict labels using the trained classifier 

pipe_1 = Pipeline([('pca', PCA(n_components = 1)),
                 ('tree', RandomForestClassifier())])

pipe_1.fit(X1_train, y1_train)

prediction_1 = pipe_1.predict(X1_test)

print('Sum of all Ones (Train) = ' + str(y1_train.sum() / y1_train.size))
print('Score (Prediction) =  ' + str(prediction_1.sum() / prediction_1.size))
print("")
print("")


Sum of all Ones (Train) = 0.5602131438721136
Score (Prediction) =  0.5241477272727273




###   
# Feature Extraction with Principal Component Analysis (PCA)
###   
## Feature Extraction 
## for Version 1: Imputed Dataset
###   

## Run PCA on whole Training Set for all possible PCAs (= number of columns)

In [ ]:
# Run PCA for all possible PCAs
pca_a1 = PCA().fit(X1_train)

# Define maximal number of principal components => the "1" in shape[1] refers to columns ("0" would be rows)
q_a1 = X1_train.shape[1]

# Get the amount of variance that each PC explains
# The eigenvalues represent the variance in the direction of the eigenvector
# These numbers for each component are proportional to the Eigenvalues 
# This means that the ratio of the eigenvalue of the first principal component 
# to the eigenvalue of the second principal component is 0.16214649
# SEE => https://stackoverflow.com/questions/37757172/finding-and-utilizing-eigenvalues-and-eigenvectors-from-pca-in-scikit-learn?rpca.q=1
expl_var_a1 = pca_a1.explained_variance_ratio_

# Get cumulative sum of the PCA 1-q_a1
sum_expl_var_a1 = np.cumsum(expl_var_a1)[:q_a1]

# because we run PCA for all possible PCAs, sum of al explained Variance of the training set should be 1

print("")
print('Explained Variance, first 10 rows: ')
print(expl_var_a1[0:10])
print("")
print('Explained Variance in Total = ' + str(expl_var_a1.sum()))
print("")
print('Cumulative explained Variance, first 10 rows: ')
print(sum_expl_var_a1[0:10])
print("")
print('Maximal number (q_1) of PCs is: ' + str(q_a1))

In [ ]:
# Plot curve with cumulative sum
plt.plot(sum_expl_var_a1)
plt.title('Cumulative explained Variance')
plt.xlabel('Number of Components')
plt.ylabel('Ratio of Cum. explained Variance')


In [ ]:
# Plot curve with explained variance
plt.plot(expl_var_a1)
plt.title('Explained Variance by single components')
plt.xlabel('Number of Components')
plt.ylabel('Ratio of Variance explained')
plt.xticks(range(-1, q_a1 + 1, 5))

In [ ]:
# Plot Feature Importances (both cumulative and idividual)
plt.figure(figsize = (12, 6))
plt.plot(expl_var_a1) #range(0, q_1 + 1), align = 'center')
plt.xticks(range(0, q_a1 + 1, 1))
plt.xlim([0, 30])
plt.xlabel('Principal Components Version a1')
plt.ylabel('Explained Variance Ratio')
plt.step(range(1, q_a1 + 1), sum_expl_var_a1, where = 'mid')

plt.tight_layout();

########  =>>> plt.bar(range(0, q_1), expl_var_1, alogn = 'center') gives ERROR MESSAGE.

### Choose number of Principal Components  and get them for further use

In [ ]:
# Define number of principal components we wish to extract
q_1 = 10

# Create PCA object
pca_1 = PCA(n_components = q_1)

# Fit PCA object to find first principal components
pca_1.fit(X1_train)

pca_1

# Get the amount of variance that each PC explains
# The eigenvalues represent the variance in the direction of the eigenvector
# These numbers for each component are proportional to the Eigenvalues 
# This means that the ratio of the eigenvalue of the first principal component 
# to the eigenvalue of the second principal component is 0.16214649
# SEE => https://stackoverflow.com/questions/37757172/finding-and-utilizing-eigenvalues-and-eigenvectors-from-pca-in-scikit-learn?rpca.q=1
expl_var_1 = pca_1.explained_variance_ratio_

# Get cumulative sum of the PCA 1-q_1
sum_expl_var_1 = np.cumsum(expl_var_1)[:q_1]

# because we run PCA for only q_1 components, sum of al explained Variance of the training set should be LESS than 1

print("")
print('Explained Variance, first 10 rows: ')
print(expl_var_1[0:10])
print("")
print('Explained Variance in Total = ' + str(expl_var_1.sum()))
print("")
print('Cumulative explained Variance, first 10 rows: ')
print(sum_expl_var_1[0:10])
print("")

### Extract q_1 number of features out of Training Set

In [ ]:
# Extract q_1 number of features according to pca analysis
# WEBSITE => https://www.analyticsvidhya.com/blog/2016/03/practical-guide-principal-component-analysis-python/
X1_train_transformed = pca_1.fit_transform(X1_train)
display(X1_train_transformed)
len(X1_train_transformed)

# This gives the Eigenvalues?

## Transform new data (Test set) using the already fitted pca_1
##    
## (((NOT SURE IT ITS CORRECT)))
##   

#### PCA components

### Dont know how to use this 

In [ ]:
# Print PCA components: every row is a principal component in the p-dimensional space
# Principal axes in feature space, representing the directions of maximum variance in the data. 
# The components are sorted by explained_variance_ 
# SEE SKLEARN DOCUMENTATION

print(pca_1.components_)

## Prediction with RandomForest

In [ ]:

X1_train_transformed = pca_1.fit_transform(X1_train)
X1_test_transformed = pca_1.transform(X1_test)


my_forest_1 = RandomForestClassifier(random_state = 1)
my_forest_1.max_depth = 8
my_forest_1.fit(X1_train_transformed, y1_train)

prediction_1 = my_forest_1.predict(X1_test_transformed)

display(prediction_1[1:5])


print('Sum of all Ones (Train) = ' + str(y1_train.sum() / y1_train.size))
print('Score (Prediction) =  ' + str(prediction_1.sum() / prediction_1.size))
print("")


# Prepare Data for Version 2: Dataset with rows dropped where Nan

### Feature Matrix and Response Vector 

In [ ]:
# Extract labels of features
labels_of_features_2 = dropnan_dataset.columns[:-1]
type(labels_of_features_2)

# X2 is the feature matrix
X2 = dropnan_dataset.iloc[:, :-1]

display(X2.head())

In [ ]:
# y2 is the response vector
y2 = dropnan_dataset.iloc[:, -1]
display(y2.head())

### Train - / Test - Split

In [ ]:
# Do the train - test- split
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size = 0.2, random_state = 0, stratify = y2)


In [ ]:
# Check if there is the approximately same percentage of '1' i both training and test response vector
display(y2_train.sum() / y2_train.size)
display(y2_test.sum() / y2_test.size)


### Standardize Variables

In [ ]:
# Standardization with sklearn StandardScaler
standard_scaler_2 = preprocessing.StandardScaler().fit(X2_train)
X2_train = standard_scaler_2.transform(X2_train)
X2_test = standard_scaler_2.transform(X2_test)


###   
## Feature Extraction 
## for Version 2: Dataset with rows dropped where Nan
###   

### Run PCA on whole Training Set for all possible PCAs (= number of columns)

In [ ]:
# Run PCA for all possible PCAs
pca_2 = PCA().fit(X2_train)

# Define maximal number of principal components => the "1" in shape[1] refers to columns ("0" would be rows)
q_2 = X2_train.shape[1]

# Get the amount of variance that each PC explains
expl_var_2 = pca_2.explained_variance_ratio_

# Get cumulative sum of the PCA 1-q
sum_expl_var_2 = np.cumsum(expl_var_2)[:q_2]


print("")
print('Explained Variance, first 10 rows: ')
print(expl_var_2[0:10])
print("")
print('Explained Variance in Total = ' + str(expl_var_2.sum()))
print("")
print('Cumulative explained Variance, first 10 rows: ')
print(sum_expl_var_2[0:10])
print("")
print('Maximal number (q_2) of PCs is: ' + str(q_2))

In [ ]:
# Plot curve with cumulative sum
plt.plot(sum_expl_var_2)
plt.title('Cumulative explained Variance')
plt.xlabel('Number of Components')
plt.ylabel('Ratio of Cum. explained Variance')


In [ ]:
# Plot curve with explained variance
plt.plot(expl_var_1)
plt.title('Explained Variance by single components')
plt.xlabel('Number of Components')
plt.ylabel('Ratio of Variance explained')
plt.xticks(range(-1, q_1 + 1, 5))

In [ ]:
# Extract q_1 number of features according to pca analysis
# WEBSITE => https://www.analyticsvidhya.com/blog/2016/03/practical-guide-principal-component-analysis-python/
X2_train_extracted = pca_2.fit_transform(X2_train)

len(X2_train_extracted)

In [ ]:
# Plot Feature Importances (both cumulative and idividual)
plt.figure(figsize = (12, 6))
plt.plot(expl_var_2) #range(0, q_2 + 1), align = 'center')
plt.xticks(range(0, q_2 + 1, 1))
plt.xlim([0, 30])
plt.xlabel('Principal Components Version 2')
plt.ylabel('Explained Variance Ratio')
plt.step(range(1, q_2 + 1), sum_expl_var_2, where = 'mid')

plt.tight_layout();

########  =>>> plt.bar(range(0, q_2), expl_var_2, alogn = 'center') gives ERROR MESSAGE.

### Choose number of Principal Components  and get them for further use

In [ ]:
# Define number of principal components we wish to extract
q_2 = 15

# Create PCA object
pca_2 = PCA(n_components = q_2)

# Fit PCA object to find first principal components
pca_2.fit(X2_train)

pca_2

# Get the amount of variance that each PC explains
# The eigenvalues represent the variance in the direction of the eigenvector
# These numbers for each component are proportional to the Eigenvalues 
# This means that the ratio of the eigenvalue of the first principal component 
# to the eigenvalue of the second principal component is 0.16214649
# SEE => https://stackoverflow.com/questions/37757172/finding-and-utilizing-eigenvalues-and-eigenvectors-from-pca-in-scikit-learn?rpca.q=1
expl_var_2 = pca_2.explained_variance_ratio_

# Get cumulative sum of the PCA 1-q_2
sum_expl_var_2 = np.cumsum(expl_var_2)[:q_2]

# because we run PCA for only q_2 components, sum of al explained Variance of the training set should be LESS than 1

print("")
print('Explained Variance, first 10 rows: ')
print(expl_var_2[0:10])
print("")
print('Explained Variance in Total = ' + str(expl_var_2.sum()))
print("")
print('Cumulative explained Variance, first 10 rows: ')
print(sum_expl_var_2[0:10])
print("")

### Extract q_2 number of features out of Training Set

In [ ]:
# Extract q_2 number of features according to pca analysis
# WEBSITE => https://www.analyticsvidhya.com/blog/2016/03/practical-guide-principal-component-analysis-python/
X2_train_extracted = pca_2.fit_transform(X2_train)
display(X2_train_extracted)
len(X2_train_extracted)

# This gives the Eigenvalues?

## Transform new data (Test set) using the already fitted pca_2
##    
## (((NOT SURE IT ITS CORRECT)))
##   

In [ ]:
# Use the fitted pca_1 on another dataset, lilke X_test to transform it
X2_test_transformed = pca_2.transform(X2_test)


#### PCA components

### Dont know how to use this 

In [ ]:
# Print PCA components: every row is a principal component in the p-dimensional space
# Principal axes in feature space, representing the directions of maximum variance in the data. 
# The components are sorted by explained_variance_ 
# SEE SKLEARN DOCUMENTATION

print(pca_2.components_)

## Pipelines and Other

In [ ]:
# https://stats.stackexchange.com/questions/144439/applying-pca-to-test-data-for-classification-purposes

